In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import scipy as sp

cap = cv2.VideoCapture("D:\\vidsnew\\vid_last.mp4")
#cap.set(cv2.CAP_PROP_POS_MSEC,350000)
cap.set(cv2.CAP_PROP_POS_MSEC,870000)
#cap = cv2.VideoCapture('vid.mp4')
#cap = cv2.VideoCapture('20160519_081947.mp4')
#cap.set(cv2.CAP_PROP_POS_MSEC,350000)
fgbg = cv2.createBackgroundSubtractorMOG2(500,50,False)
perFrame=False
orb = cv2.ORB_create()
prevBlack=np.zeros((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),3), np.uint8)
img3=np.zeros((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),3), np.uint8)
img2=np.zeros((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),3), np.uint8)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
i = 0
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
kernel1 = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
while(1):
    i+=1
    ret, frame = cap.read()
    black = np.zeros((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),3), np.uint8)
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    res = cv2.bitwise_and(frame,frame,mask = fgmask)
    #edges = cv2.Canny(fgmask,200,200)
    fgmask = cv2.dilate(fgmask, kernel1, iterations=10)

    ret,thresh = cv2.threshold(fgmask,127,255,cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    hsv = cv2.cvtColor(res, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([20,50,50])
    upper_yellow = np.array([50,255,255])
    maskcol = cv2.inRange(hsv, lower_yellow, upper_yellow)
    res1 = cv2.bitwise_and(res,res,mask = maskcol)
    
    #cv2.putText(frame,str(perFrame), (100,100), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    if(len(contours)<1e3):
        resCnt=[]
        for elem in contours:
                area=cv2.contourArea(elem)
                if(area>3):
                    x,y,w,h = cv2.boundingRect(elem)
                    rect=frame[y:y+h, x:x+w]
                    
                    black[y:y+h, x:x+w]=rect
                    #ellipse = cv2.fitEllipse(elem)
                    #cv2.ellipse(frame,ellipse,(0,255,0),2)
                    (x,y),radius = cv2.minEnclosingCircle(elem)
                    center = (int(x),int(y))
                    radius = int(radius)
                    #cv2.circle(frame,center,radius,(255,255,255),1)
                    resCnt.append(elem)
                    
                #cv2.drawContours(black,[box],0,(0,0,255),2)        
    #imageG=cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    #ret,thresh = cv2.threshold(imageG,127,255,cv2.THRESH_BINARY)
    #im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    for elem in contours:
        area=cv2.contourArea(elem)
        if(area>3):
            mask = np.zeros(res1.shape[:2], dtype="uint8")
            cv2.drawContours(mask, [elem], -1, 255, -1)
            #mask = cv2.erode(mask, None, iterations=2)
            mean = cv2.mean(hsv, mask=mask)[:3]
            M = cv2.moments(elem)
            #print(str(elem))
            if(M["m00"]!=0 and M["m00"]!=0):
                cX = int((M["m10"] / M["m00"]))
                cY = int((M["m01"] / M["m00"]))
                cv2.putText(res,str(mean), (cX,cY), cv2.FONT_HERSHEY_SIMPLEX, 0.3, 255)
    cv2.drawContours(hsv, contours, -1, (255,255,255), 1)
    #cv2.drawKeypoints(black,kp2,img3,color=(0,255,0), flags=0)
    #cv2.drawKeypoints(prevBlack,kp1,img2,color=(0,255,0), flags=0)
    #cv2.imshow('asdasd',frame)
    cv2.imshow('frame',res1)
    cv2.imshow('asdasdf',res)
    cv2.imshow('asdasdff',frame)
    #cv2.imshow('im3',img3)
    #prevBlack=black
    
    delay = 1
    k = cv2.waitKey(delay)
    if(k!=-1):
        print (k)
    if k == 27:
        break
    if k == 112:
        cv2.imwrite('pic{:>05}.jpg'.format(i), res)
    if k == 32 or perFrame==True:
        k = cv2.waitKey(0)
        if(k==32):
            perFrame=False
            continue
        if k == 112:
            cv2.imwrite('pic{:>05}.jpg'.format(i), res)
            cv2.imwrite('picr{:>05}.jpg'.format(i), res1)
        if(k == 2555904): 
            perFrame=True
            continue
    
        
    

cap.release()
cv2.destroyAllWindows()